In [16]:
"""
Rules-based Intent Router 
------------------------------------------------
Flow:
#0) Setup/Config → RULES_PATH, THRESHOLD
#1A) Rules I/O → load_rules, save_rules
#1B) Normalize + RULES init
#2) Mock functions (replace with real APIs later)
#3A) predict_intent (keyword score)
#3B) run_pipeline (fallback → route → call)
#4) ask(...) to log gold labels in EVAL_ROWS
#5) improve_from_logs to propose/apply keywords (self-improvement)
#6) Mini demo: ask → preview → apply → save

"""

'\nRules-based Intent Router \n------------------------------------------------\nFlow:\n#0) Setup/Config → RULES_PATH, THRESHOLD\n#1A) Rules I/O → load_rules, save_rules\n#1B) Normalize + RULES init\n#2) Mock functions (replace with real APIs later)\n#3A) predict_intent (keyword score)\n#3B) run_pipeline (fallback → route → call)\n#4) ask(...) to log gold labels in EVAL_ROWS\n#5) improve_from_logs to propose/apply keywords (self-improvement)\n#6) Mini demo: ask → preview → apply → save\n\n'

In [17]:
# === Setup & Config ===
from __future__ import annotations   # Allow postponed evaluation of type annotations (PEP 563/649)

# --- Standard library imports ---
import json          # JSON read/write for rules storage
import re            # Regular expressions (text normalization, keyword matching)
import copy          # Deep/shallow copy utilities (used when updating rules safely)
import collections   # Counter, defaultdict (used for keyword frequency analysis)
import random        # Randomization (tie-breaking, shuffling candidates)
import tempfile      # Create temporary files (atomic save of rules.json)
import time          # Time-based IDs for mock functions (ticket/order IDs)
from pathlib import Path  # Object-oriented file system paths
from typing import Any, Dict, List, Optional, Tuple  # Type hints for clarity

In [18]:
# Confidence threshold: below this → fallback
THRESHOLD: float = 1

In [19]:
# === Step 1-A) Rules I/O: load_rules / save_rules ===

In [20]:
# Where to store/load rule definitions
from pathlib import Path  
RULES_PATH: Path = Path("rules.json")

In [25]:
   """
    Load keyword-based intent rules from disk.

    - If the file exists:
        → Read it as UTF-8 JSON and return as a dictionary.
    - If the file does not exist or fails to load:
        → Return a default dictionary with a few fallback intents.
    """

'\n Load keyword-based intent rules from disk.\n\n - If the file exists:\n     → Read it as UTF-8 JSON and return as a dictionary.\n - If the file does not exist or fails to load:\n     → Return a default dictionary with a few fallback intents.\n '

In [27]:
def load_rules(path: Path = RULES_PATH) -> Dict[str, Dict[str, List[str]]]:
    if path.exists():
        try:
            return json.loads(path.read_text(encoding="utf-8"))
        except Exception as e:
            print(f"⚠️ Failed to load {path}: {e}. Using defaults...")

    # --- Default fallback rules (safe starting point) ---
    return {
        "report_card_lost": {
            "keywords": ["lost my card", "card stolen", "card missing", "분실", "도난"],
            "args": ["user_id"],
        },
        "check_card_delivery": {
            "keywords": ["track my card", "where is my card", "card not arrived", "배송", "언제 도착"],
            "args": ["card_id"],
        },
        "request_new_card": {
            "keywords": ["order new card", "replace my card", "new card", "재발급"],
            "args": ["user_id"],
        },
    }

def save_rules(rules: Dict[str, Any], path: Path = RULES_PATH) -> None:
    """
    Save the given rules back to disk as pretty UTF-8 JSON.
    """
    try:
        path.write_text(json.dumps(rules, ensure_ascii=False, indent=2), encoding="utf-8")
        print(f"💾 Rules saved to {path}")
    except Exception as e:
        print(f"⚠️ Failed to save rules: {e}")


In [28]:
# === Quick test for Step 1-A ===

# Load rules (either from file or defaults)
rules = load_rules()
print("✅ Loaded rules:")
print(json.dumps(rules, indent=2, ensure_ascii=False))  # pretty print

# Save rules back to disk
save_rules(rules)

# Reload to confirm it's written correctly
reloaded = load_rules()
print("✅ Reloaded rules (from file):")
print(json.dumps(reloaded, indent=2, ensure_ascii=False))


✅ Loaded rules:
{
  "report_card_lost": {
    "keywords": [
      "lost my card",
      "card stolen",
      "card missing",
      "분실",
      "도난",
      "card",
      "missing last",
      "night",
      "yesterday",
      "lost",
      "card yesterday",
      "misplaced",
      "went",
      "신고해 주세요"
    ],
    "args": [
      "user_id"
    ]
  },
  "check_card_delivery": {
    "keywords": [
      "track my card",
      "where is my card",
      "card not arrived",
      "배송",
      "언제 도착",
      "card",
      "expect card",
      "when should",
      "arrived",
      "도착하나요",
      "card hasn",
      "delivery",
      "card delivery",
      "should",
      "hasn arrived",
      "where card",
      "where"
    ],
    "args": [
      "card_id"
    ]
  },
  "request_new_card": {
    "keywords": [
      "order new card",
      "replace my card",
      "new card",
      "재발급",
      "card",
      "physical card",
      "new",
      "재발급 부탁드립니다",
      "get replacement",
      "get",
 

In [31]:
#Step 1-B — Normalizer + RULES

In [ ]:
# === Small normalizer for naive keyword matching ===
# - Converts text to lowercase
# - Keeps only digits, Latin letters, Korean characters, and spaces
# - Collapses multiple spaces into a single space
_WS = re.compile(r"\s+")

In [29]:
import re
from typing import Any, Dict

# Helper regex: collapse multiple spaces into one
_WS = re.compile(r"\s+")

def normalize_text(s: str) -> str:
    """
    Very simple text normalization:
    - Lowercase everything
    - Remove punctuation/symbols (keep letters, numbers, Korean)
    - Collapse multiple spaces into one
    """
    s = s.lower()
    s = re.sub(r"[^0-9a-z가-힣\s]", " ", s)  # keep only letters/numbers/Korean
    s = _WS.sub(" ", s).strip()
    return s

# === Initialize RULES once at startup ===
# RULES will be updated later during self-improvement
RULES: Dict[str, Any] = load_rules()
print("✅ Loaded intents:", list(RULES.keys()))

✅ Loaded intents: ['report_card_lost', 'check_card_delivery', 'request_new_card']


In [30]:
# === Step 2) Mock functions (replace with real APIs later) ===
import time
from typing import Any, Dict

In [31]:
def report_card_lost(user_id: str) -> Dict[str, Any]:
    """
    Mock function: simulate reporting a lost card.

    - Input: user_id (str)
    - Process: make a fake ticket_id using current time
    - Output: dict {ok, ticket_id, user_id}
    """
    ticket_id = f"T{int(time.time()) % 100000:05d}"  # 5-digit ID from time
    return {"ok": True, "ticket_id": ticket_id, "user_id": user_id}

In [32]:
def check_card_delivery(card_id: str) -> Dict[str, Any]:
    """
    Mock function: simulate checking the delivery status of a card.

    - Input: card_id (str)
    - Output: dict {ok, card_id, carrier, eta, last_update}
    """
    return {
        "ok": True,
        "card_id": card_id,
        "carrier": "UPS",                 # pretend shipping company
        "eta": "3-5 business days",       # fake delivery time
        "last_update": "out for delivery" # fake status
    }

In [33]:
def request_new_card(user_id: str) -> Dict[str, Any]:
    """
    Mock function: simulate placing a new card order.

    - Input: user_id (str)
    - Process: make a fake order_id using current time
    - Output: dict {ok, order_id, user_id}
    """
    order_id = f"O{int(time.time()) % 100000:05d}"  # 5-digit ID from time
    return {"ok": True, "order_id": order_id, "user_id": user_id}

In [34]:
# === Step 2 sanity check ===

print("Test report_card_lost:")
print(report_card_lost("U123"))

print("\nTest check_card_delivery:")
print(check_card_delivery("C5555"))

print("\nTest request_new_card:")
print(request_new_card("U999"))


Test report_card_lost:
{'ok': True, 'ticket_id': 'T21574', 'user_id': 'U123'}

Test check_card_delivery:
{'ok': True, 'card_id': 'C5555', 'carrier': 'UPS', 'eta': '3-5 business days', 'last_update': 'out for delivery'}

Test request_new_card:
{'ok': True, 'order_id': 'O21574', 'user_id': 'U999'}


In [35]:
#Step 3-A Intent detection (pure function, no side effects)

In [59]:
    """
    Predict the most likely intent from the given text.

    Args:
        text (str): User input message.
        rules (dict): Dictionary of intent rules, 
                      each intent has "keywords" list.

    Returns:
        (intent_name, confidence):
            - intent_name (str): best matching intent, or "nlu_fallback" if none
            - confidence (float): naive confidence score between 0 and 1

    Confidence is calculated as:
        (# of matched keywords) / (# of keywords for that intent)

    Notes:
    - Very simple keyword-based scoring.
    - Only counts exact normalized keyword matches.
    - If no keywords match, returns ("nlu_fallback", 0.0).
    """

'\nPredict the most likely intent from the given text.\n\nArgs:\n    text (str): User input message.\n    rules (dict): Dictionary of intent rules, \n                  each intent has "keywords" list.\n\nReturns:\n    (intent_name, confidence):\n        - intent_name (str): best matching intent, or "nlu_fallback" if none\n        - confidence (float): naive confidence score between 0 and 1\n\nConfidence is calculated as:\n    (# of matched keywords) / (# of keywords for that intent)\n\nNotes:\n- Very simple keyword-based scoring.\n- Only counts exact normalized keyword matches.\n- If no keywords match, returns ("nlu_fallback", 0.0).\n'

In [38]:
#Main intent predictor

In [37]:
from typing import Tuple

def predict_intent(text: str, rules: Dict[str, Any]) -> Tuple[str, float]:

    # Normalize the input text and pad with spaces
    # → ensures keywords like "card" are matched as whole words
    t = " " + normalize_text(text) + " "
    
    # Default result = fallback
    best: Tuple[str, float] = ("nlu_fallback", 0.0)

    # Iterate over each intent and compute match score
    for intent, cfg in rules.items():
        kws: List[str] = cfg.get("keywords", [])
        hit = 0

        # Count how many keywords appear in the text
        for kw in kws:
            kw_norm = " " + normalize_text(kw) + " "
            if kw_norm and kw_norm in t:
                hit += 1

        # Confidence = ratio of hits to total keywords
        conf = min(1.0, hit / max(1, len(kws))) if kws else 0.0

        # Keep the best scoring intent
        if conf > best[1]:
            best = (intent, conf)

    return best

In [39]:
# === Quick test for predict_intent ===

# 1) Example rules (or you can use load_rules())
rules = {
    "report_card_lost": {
        "keywords": ["lost my card", "card stolen", "card missing", "분실", "도난"]
    },
    "check_card_delivery": {
        "keywords": ["track my card", "where is my card", "card not arrived", "배송", "언제 도착"]
    },
    "request_new_card": {
        "keywords": ["order new card", "replace my card", "new card", "재발급"]
    },
}

# 2) Example test sentences
tests = [
    "I lost my card yesterday",
    "Can you track my card delivery?",
    "Please issue me a new card",
    "When will my card arrive?",
    "What is the weather today?"  # should fallback
]

# 3) Run predictions
for t in tests:
    intent, conf = predict_intent(t, rules)
    print(f"Input: {t}")
    print(f" → Predicted intent: {intent}, confidence={conf:.2f}\n")

Input: I lost my card yesterday
 → Predicted intent: report_card_lost, confidence=0.20

Input: Can you track my card delivery?
 → Predicted intent: check_card_delivery, confidence=0.20

Input: Please issue me a new card
 → Predicted intent: request_new_card, confidence=0.25

Input: When will my card arrive?
 → Predicted intent: nlu_fallback, confidence=0.00

Input: What is the weather today?
 → Predicted intent: nlu_fallback, confidence=0.00



In [37]:
#Detect intent and handle fallback

In [46]:
def run_pipeline(
    text: str,
    user_id: Optional[str] = None,
    card_id: Optional[str] = None,
) -> Dict[str, Any]:
    """
    End-to-end pipeline:
    1) Detect intent
    2) Handle fallback
    3) Handle each intent (lost card / check delivery / request new card)
    """

    # Step 1: Detect intent
    intent, conf = predict_intent(text, RULES)

    # Step 2: Fallback
    if conf < THRESHOLD:
        return {
            "message": "Sorry, I couldn't understand. Please rephrase.",
            "intent": "nlu_fallback",
            "confidence": conf,
            "called": None,
            "result": None,
        }

    # =========================
    # Block 1 — report_card_lost
    # =========================
    if intent == "report_card_lost":
        if not user_id:
            return {
                "message": "Please provide your user_id.",
                "intent": intent,
                "confidence": conf,
                "called": None,
                "result": None,
            }
        res = report_card_lost(user_id)
        return {
            "message": f"I've created a lost-card ticket: {res['ticket_id']}.",
            "intent": intent,
            "confidence": conf,
            "called": "report_card_lost",
            "result": res,
        }

    # =========================
    # Block 2 — check_card_delivery
    # =========================
    if intent == "check_card_delivery":
        if not card_id:
            return {
                "message": "Please provide your card_id.",
                "intent": intent,
                "confidence": conf,
                "called": None,
                "result": None,
            }
        res = check_card_delivery(card_id)
        return {
            "message": f"Your card is with {res['carrier']}, ETA {res['eta']} (status: {res['last_update']}).",
            "intent": intent,
            "confidence": conf,
            "called": "check_card_delivery",
            "result": res,
        }

    # =========================
    # Block 3 — request_new_card
    # =========================
    if intent == "request_new_card":
        if not user_id:
            return {
                "message": "Please provide your user_id.",
                "intent": intent,
                "confidence": conf,
                "called": None,
                "result": None,
            }
        res = request_new_card(user_id)
        return {
            "message": f"Your new card request has been placed. Order ID: {res['order_id']}.",
            "intent": intent,
            "confidence": conf,
            "called": "request_new_card",
            "result": res,
        }


In [48]:
# === Step 3 sanity check (Block 1) ===
# Define test cases for run_pipeline (English only)

test_cases = [
    {"text": "I lost my card yesterday", "user_id": "U123"},             # lost card
    {"text": "Where is my card? It hasn't arrived.", "card_id": "C5555"},# delivery check
    {"text": "Please order a new card", "user_id": "U999"},              # new card request
    {"text": "I need to report my card as lost", "user_id": "K1001"},    # lost card (English version)
    {"text": "When will my card be delivered?", "card_id": "K-9999"},    # delivery check (English version)
    {"text": "Hello, can you help me?"},                                 # fallback expected
]

In [49]:
# === Step 3 sanity check (Block 2) ===
# Run pipeline for each test case and print results

for case in test_cases:
    text = case.get("text", "")
    user_id = case.get("user_id")
    card_id = case.get("card_id")

    result = run_pipeline(text, user_id=user_id, card_id=card_id)

    print("USER :", text)
    print("ASSIST:", result["message"],
          f"(intent={result['intent']}, conf={result['confidence']:.2f}, called={result['called']})")
    print("-" * 60)

USER : I lost my card yesterday
ASSIST: Sorry, I couldn't understand. Please rephrase. (intent=nlu_fallback, conf=0.36, called=None)
------------------------------------------------------------
USER : Where is my card? It hasn't arrived.
ASSIST: Sorry, I couldn't understand. Please rephrase. (intent=nlu_fallback, conf=0.24, called=None)
------------------------------------------------------------
USER : Please order a new card
ASSIST: Sorry, I couldn't understand. Please rephrase. (intent=nlu_fallback, conf=0.33, called=None)
------------------------------------------------------------
USER : I need to report my card as lost
ASSIST: Sorry, I couldn't understand. Please rephrase. (intent=nlu_fallback, conf=0.14, called=None)
------------------------------------------------------------
USER : When will my card be delivered?
ASSIST: Sorry, I couldn't understand. Please rephrase. (intent=nlu_fallback, conf=0.08, called=None)
------------------------------------------------------------
USER

In [50]:
#Step 4 — Evaluate assistant by logging gold labels

In [51]:
STOP = {
    "the","a","an","to","for","of","is","are","it","my","your","please","and",
    "좀","해주세요","해줘요","합니다","요","이","그","저","은","는","이요","에","가",
}

def evaluate(
    rows: List[Dict[str, str]],
    rules: Dict[str, Any],
    threshold: float = 0.0,
    verbose: bool = False,
) -> Tuple[float, List[Dict[str, Any]]]:
    """rows = [{"text": ..., "label": ...}, ...]"""
    ok, mistakes = 0, []
    for r in rows:
        pred, conf = predict_intent(r["text"], rules)
        if conf < threshold:
            pred = "nlu_fallback"
        good = (pred == r["label"])
        ok += int(good)
        if not good:
            mistakes.append({"text": r["text"], "gold": r["label"], "pred": pred, "conf": round(conf, 3)})
        if verbose:
            print(f"[pred] {pred} ({conf:.2f}) | gold={r['label']} | {r['text']}")
    acc = ok / max(1, len(rows))
    return acc, mistakes


def extract_candidates(text: str, minlen: int = 3, max_ngram: int = 2) -> List[str]:
    """Take tokens + bigrams from one text (very naive)."""
    t = normalize_text(text)
    toks = [w for w in t.split() if len(w) >= minlen and w not in STOP]
    cands = toks[:]
    if max_ngram >= 2:
        for i in range(len(toks) - 1):
            cands.append(toks[i] + " " + toks[i + 1])
    return cands


def propose_keywords(
    mistakes: List[Dict[str, Any]],
    rules: Dict[str, Any],
    topk: int = 3,
    minlen: int = 3,
) -> Dict[str, List[str]]:
    """Collect frequent tokens/ngrams per gold label, excluding existing keywords."""
    by_label: Dict[str, List[str]] = collections.defaultdict(list)
    for m in mistakes:
        by_label[m["gold"]].append(m["text"])

    proposals: Dict[str, List[str]] = {}
    for label, texts in by_label.items():
        freq: collections.Counter[str] = collections.Counter()
        existing = {normalize_text(k) for k in rules.get(label, {}).get("keywords", [])}
        for txt in texts:
            for cand in extract_candidates(txt, minlen=minlen):
                if normalize_text(cand) not in existing:
                    freq[cand] += 1
        items = list(freq.items())
        random.shuffle(items)  # tie-breaker
        items.sort(key=lambda x: x[1], reverse=True)
        proposals[label] = [w for w, c in items[:topk]]
    return proposals


def improve_from_logs(apply: bool = False, topk: int = 3, minlen: int = 3) -> None:
    """
    1) Evaluate current RULES on EVAL_ROWS
    2) Print suggested keywords per label
    3) (Optional) apply suggestions to RULES and re-evaluate
    """
    global RULES
    acc0, mistakes0 = evaluate(EVAL_ROWS, RULES, threshold=THRESHOLD)
    print(f"[before] accuracy={acc0:.4f} total={len(EVAL_ROWS)} mistakes={len(mistakes0)}")

    props = propose_keywords(mistakes0, RULES, topk=topk, minlen=minlen)
    print("\n== SUGGESTED KEYWORDS ==")
    for lab, kws in props.items():
        print(f"- {lab}: {kws}")

    if not apply:
        print("\n(Preview only — set apply=True to update RULES)")
        return

    new_rules = copy.deepcopy(RULES)
    for lab, kws in props.items():
        new_rules.setdefault(lab, {}).setdefault("keywords", [])
        existing = set(new_rules[lab]["keywords"])
        for k in kws:
            if k not in existing:
                new_rules[lab]["keywords"].append(k)

    RULES = new_rules
    acc1, mistakes1 = evaluate(EVAL_ROWS, RULES, threshold=THRESHOLD)
    print(f"\n[after ] accuracy={acc1:.4f} total={len(EVAL_ROWS)} mistakes={len(mistakes1)} Δ={acc1-acc0:+.4f}")


In [52]:
# 1) Interact & log gold labels
ask("I lost my card yesterday", user_id="U1001", gold_intent="report_card_lost")
ask("Where is my card? It hasn't arrived.", card_id="C5555", gold_intent="check_card_delivery")
ask("Please order a new card", user_id="U1001", gold_intent="request_new_card")
ask("분실 신고 좀 해주세요", user_id="U1002", gold_intent="report_card_lost")
ask("배송 언제 도착하나요?", card_id="K-9999", gold_intent="check_card_delivery")

# 2) See suggestions from the mistakes we logged (no apply yet)
improve_from_logs(apply=False, topk=3)

# 3) Apply suggestions → RULES updated in-place → re-evaluate
improve_from_logs(apply=True, topk=3)

# 4) (optional) Save updated rules to disk for next session
save_rules(RULES)


NameError: name 'ask' is not defined

In [15]:
# Ambiguous / paraphrased / mixed-language / negatives
ask("misplaced my bank card", user_id="U2001", gold_intent="report_card_lost")
ask("my card went missing last night", user_id="U2002", gold_intent="report_card_lost")
ask("카드 잃어버렸어요 신고해 주세요", user_id="U2003", gold_intent="report_card_lost")

ask("has my card shipped yet?", card_id="C7777", gold_intent="check_card_delivery")
ask("when should I expect the card delivery", card_id="C8888", gold_intent="check_card_delivery")
ask("카드 언제 오나요? 아직 못 받았어요", card_id="C9999", gold_intent="check_card_delivery")

ask("get me a replacement please", user_id="U3001", gold_intent="request_new_card")
ask("need a new physical card", user_id="U3002", gold_intent="request_new_card")
ask("카드 재발급 부탁드립니다", user_id="U3003", gold_intent="request_new_card")

# “잡담/무관” → 의도 없음(= fallback이 맞음)
ask("what's the weather today?", gold_intent="nlu_fallback")
ask("recommend me a movie", gold_intent="nlu_fallback")

# 인자 누락(일부러): 올바른 intent지만 파라미터가 없어야 'Please provide ...' 메시지
ask("I lost my card", gold_intent="report_card_lost")           # user_id 없음
ask("track my card delivery", gold_intent="check_card_delivery")# card_id 없음


USER : misplaced my bank card
ASSIST: Sorry, I couldn't understand. Could you rephrase? (intent=nlu_fallback, conf=0.14)
[logged gold=report_card_lost]
USER : my card went missing last night
ASSIST: Sorry, I couldn't understand. Could you rephrase? (intent=nlu_fallback, conf=0.27)
[logged gold=report_card_lost]
USER : 카드 잃어버렸어요 신고해 주세요
ASSIST: Sorry, I couldn't understand. Could you rephrase? (intent=nlu_fallback, conf=0.00)
[logged gold=report_card_lost]
USER : has my card shipped yet?
ASSIST: Sorry, I couldn't understand. Could you rephrase? (intent=nlu_fallback, conf=0.14)
[logged gold=check_card_delivery]
USER : when should I expect the card delivery
ASSIST: Sorry, I couldn't understand. Could you rephrase? (intent=nlu_fallback, conf=0.18)
[logged gold=check_card_delivery]
USER : 카드 언제 오나요? 아직 못 받았어요
ASSIST: Sorry, I couldn't understand. Could you rephrase? (intent=nlu_fallback, conf=0.00)
[logged gold=check_card_delivery]
USER : get me a replacement please
ASSIST: Sorry, I couldn'

{'message': "Sorry, I couldn't understand. Could you rephrase?",
 'intent': 'nlu_fallback',
 'confidence': 0.18181818181818182,
 'called': None,
 'result': None}

In [16]:
improve_from_logs(apply=False, topk=3)  # 어디가 약한지 제안만 보기
improve_from_logs(apply=True,  topk=3)  # 규칙 업데이트 + 재평가
save_rules(RULES)                       # rules.json에 반영


[before] accuracy=0.2778 total=18 mistakes=13

== SUGGESTED KEYWORDS ==
- check_card_delivery: ['card delivery', 'delivery', 'should expect']
- report_card_lost: ['잃어버렸어요', 'went', '잃어버렸어요 신고해']
- request_new_card: ['replacement', 'get replacement', 'get']

(Preview only — set apply=True to update RULES)
[before] accuracy=0.2778 total=18 mistakes=13

== SUGGESTED KEYWORDS ==
- check_card_delivery: ['delivery', 'card delivery', 'should']
- report_card_lost: ['misplaced', 'went', '신고해 주세요']
- request_new_card: ['재발급 부탁드립니다', 'get replacement', 'get']

[after ] accuracy=0.2222 total=18 mistakes=14 Δ=-0.0556
